In [2]:
#ライブラリのインポート
#tkinter系ライブラリ
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk

#ディープラーニング系ライブラリ
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout

#その他必要なライブラリ
import codecs
import glob,os,re,time
import numpy as np
import os
import pandas as pd
import time
import traceback


In [21]:
class DataX:
    def __init__(self):
        self.cnt_x = 0
        self.cnt_fnfe = 0
        self.cnt_str = 0

    def set(self): 
        # ファイルダイアログメニューの設定
        typ = [("エクセルファイル","*.xlsx")]
        file_path = filedialog.askopenfilename(filetypes = typ)
        file_name = os.path.split(file_path)[1]

        try:
            box1.delete(0, tk.END)
            df_x = pd.read_excel(file_path, index_col=0)
            self.df_x = df_x
            columns = df_x.columns.values.tolist()
            self.columns = columns
            self.cnt_x = 0
            self.cnt_fnfe = 0
            self.cnt_str = 0

            if df_x.index[0] != 0:
                messagebox.showerror('警告', "列名の中にnullが存在します。")
                self.cnt_x += 1
        
            for i in range(len(columns)):
                for row_is_str in df_x[columns[i]].apply(lambda x: isinstance(x, str)):
                    if row_is_str is True:
                       self.cnt_str += 1 
                
                if (self.cnt_str != 0) and df_x[columns[i]].isnull().values.any():
                    messagebox.showerror('警告', f'データの {columns[i]}列 に文字列とNullが含まれていました。データを見直して下さい。')
                    self.cnt_x += 1
                    self.cnt_str = 0                
                    
                elif (self.cnt_str != 0) :
                    messagebox.showerror('警告', f'データの {columns[i]}列 に文字列が含まれていました。該当列データを見直して下さい。')
                    self.cnt_x += 1
                    self.cnt_str = 0
                
                elif df_x[columns[i]].isnull().values.any():
                    messagebox.showerror('警告', f'データの {columns[i]}列 にNullが含まれていました。該当列データを見直して下さい。')
                    self.cnt_x += 1

        except UnicodeDecodeError as e:
            t_x = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "データの文字コードをutf-8に変更して下さい。\n"+t_x[0])
            self.cnt_x += 1
            
        except FileNotFoundError as e:
            t_x = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "ファイルが選択されていません。\n"+t_x[0])
            #変数を初期化する必要がある
            self.cnt_x += 1
            self.cnt_fnfe += 1
            try:
                del self.df_x
            except AttributeError as e:
                t_x = traceback.format_exception_only(type(e), e)
                if "DataX" in t_x[0]:
                    self.cnt_x += 1
                    
        if self.cnt_x == 0:
            box1.delete(0, tk.END)
            box1.insert(tk.END, f"「{file_name}」ファイルが正しく挿入されました。")  

        elif self.cnt_fnfe != 0:
            box1.delete(0, tk.END)
            box1.insert(tk.END, "ファイルが選択されていません。")
            self.cnt_fnfe = 0
            
        else:
            box1.delete(0, tk.END)
            box1.insert(tk.END, "【エラー】データを見直して下さい。")   
            self.cnt_x = 0


In [107]:
class DataY:
    def __init__(self):
        self.cnt_y = 0
        self.cnt_fnfe_y = 0
        self.cnt_str_y = 0

    def set(self): 
        
        # ファイルダイアログメニューの設定
        typ = [("エクセルファイル","*.xlsx")]
        file_path_y = filedialog.askopenfilename(filetypes = typ)
        file_name = os.path.split(file_path_y)[1]
        
        try:
            box2.delete(0, tk.END)
            df_y = pd.read_excel(file_path_y, index_col=0)
            df_x = data_x.df_x
            self.df_y = df_y
            columns_y = self.df_y.columns.values.tolist()
            self.columns_y = columns_y
            self.cnt_y = 0
            self.cnt_fnfe_y = 0
            self.cnt_str_y = 0
            
            if df_y.shape[0] != df_x.shape[0]:
                messagebox.showerror('警告', "説明変数データXと行数が一致していません。")
                self.cnt_y += 1
            
            if df_y.index[0] != 0:
                messagebox.showerror('警告', "列名の中にnullが存在します。")
                self.cnt_y += 1
                    
            if len(self.columns_y) != 1:
                messagebox.showerror("警告", "データの列数は一つである必要があります。")
                self.cnt_y += 1
                
            else:
                
                for row_is_str in df_y[columns_y[0]].apply(lambda x: isinstance(x, str)):
                    if row_is_str is True:
                        self.cnt_str_y += 1 
                        
                if (self.cnt_str_y != 0) and df_y[columns_y[0]].isnull().values.any():
                    messagebox.showerror("警告", "データに文字列とNullが含まれていました。データを見直して下さい。")
                    self.cnt_y += 1
                    self.cnt_str_y = 0                
                    
                elif (self.cnt_str_y != 0) :
                    messagebox.showerror('警告', "データに文字列が含まれていました。該当列データを見直して下さい。")
                    self.cnt_y += 1
                    self.cnt_str_y = 0
        
                elif df_y[columns_y[0]].isnull().values.any():
                    messagebox.showerror('警告', "データにNullが含まれていました。該当列データを見直して下さい。")
                    self.cnt_y += 1

        except UnicodeDecodeError as e:
            t_y = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "データの文字コードをutf-8に変更して下さい。\n"+t_y[0])
            self.cnt_y += 1

        except FileNotFoundError as e:
            t_y = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "ファイルが選択されていません。\n"+t_y[0])
            #変数を初期化する必要がある
            self.cnt_y += 1
            self.cnt_fnfe_y += 1
            try:
                del self.df_y
            except AttributeError as e:
                t_y = traceback.format_exception_only(type(e), e)
                if "DataY" in t_y[0]:
                    self.cnt_y += 1
            except NameError as e:
                t_y = traceback.format_exception_only(type(e), e)
                if "DataY" in t_y[0]:
                    self.cnt_y += 1

        except NameError as e:
            t_y = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "説明変数データXが入力されていません。\n"+t_y[0])
            self.cnt_y += 1

        except AttributeError as e:
            t_y = traceback.format_exception_only(type(e), e)
            if "DataX" in t_y[0]:
                messagebox.showerror('警告', "説明変数Xが選択されていません\n"+t_y[0])
                self.cnt_y += 1
                    
        if self.cnt_y == 0:
            box2.delete(0, tk.END)
            box2.insert(tk.END, f"「{file_name}」ファイルが正しく挿入されました。")  

        elif self.cnt_fnfe_y != 0:
            box2.delete(0, tk.END)
            box2.insert(tk.END, "ファイルが選択されていません")
            self.cnt_fnfe_y = 0
            
        else:
            box2.delete(0, tk.END)
            box2.insert(tk.END, "【エラー】データを見直して下さい。")
            self.cnt_y = 0
            

In [97]:
class DataX2:
    def __init__(self):
        self.cnt2 = 0
        self.cnt_fnfe_x2 = 0
        self.cnt_str2 = 0
        self.cnt_set_x2 = 0  

    def set(self): 
        
        # ファイルダイアログメニューの設定
        typ = [("エクセルファイル","*.xlsx")]
        file_path2 = filedialog.askopenfilename(filetypes = typ)
        file_name = os.path.split(file_path2)[1]

        try:
            box7.delete(0, tk.END)
            df_x2 = pd.read_excel(file_path2,index_col=0)
            columns_x2 = df_x2.columns.values.tolist()
            columns_x = data_x.columns
            df_x = data_x.df_x
            df_y = data_y.df_y
            self.cnt2 = 0                
            self.cnt_str2 = 0
            self.cnt_set_x2 = 0
            self.cnt_fnfe_x2 = 0 #何列も違う列名が存在した場合表示を一回にするためのカウンタ変数
            
            #これいるのか？
            if df_x2.shape[0] != df_x.shape[0]:
                messagebox.showerror('警告', "説明変数データXと行数が一致しないか、データXが存在していません。")
                self.cnt2 += 1
                            
            # if df_x2.shape[0] != df_y.shape[0]:       
            #     messagebox.showerror('警告', "目的変数データYと行数が一致しないか、データYが存在していません。")
            #     self.cnt2 += 1
                
            if df_x2.index[0] != 0:
                messagebox.showerror('警告', "列名の中にnullが存在します。")
                self.cnt2 += 1
                
            for i in range(len(columns_x2)):
                for row_is_str in df_x2[columns_x2[i]].apply(lambda x: isinstance(x, str)):
                    if row_is_str is True:
                       self.cnt_str2 += 1 
                
                if (self.cnt_str2 != 0) and df_x2[columns_x2[i]].isnull().values.any():
                    messagebox.showerror('警告', f'データの {columns_x2[i]}列 に文字列とNullが含まれていました。データを見直して下さい。')
                    self.cnt2 += 1
                    self.cnt_str2 = 0                
                    
                elif (self.cnt_str2 != 0) :
                    messagebox.showerror('警告', f'データの {columns_x2[i]}列 に文字列が含まれていました。該当列データを見直して下さい。')
                    self.cnt2 += 1
                    self.cnt_str2 = 0
                
                elif df_x2[columns_x2[i]].isnull().values.any():
                    messagebox.showerror('警告', f'データの {columns_x2[i]}列 にNullが含まれていました。該当列データを見直して下さい。')
                    self.cnt2 += 1
        
                elif set(columns_x) != set(columns_x2) and self.cnt_set_x2 == 0:
                    messagebox.showerror(tk.END, "学習用データと列名が一致しません。データを見直して下さい。")
                    self.cnt2 += 1
                    self.cnt_set_x2 += 1

        except UnicodeDecodeError as e:
            t_x2 = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "データの文字コードをutf-8に変更して下さい。\n"+t_x2[0])
            self.cnt2 += 1 
            
        except FileNotFoundError as e:
            t_x2 = traceback.format_exception_only(type(e), e)
            messagebox.showerror('警告', "ファイルが選択されていません。\n"+t_x2[0])
            self.cnt2 += 1
            self.cnt_fnfe_x2 += 1
            try:
                del self.df_x2
            except AttributeError as e:
                t_x2 = traceback.format_exception_only(type(e), e)
                if "DataX2" in t_x2[0]:
                    self.cnt2 += 1
            except NameError as e:
                t_x2 = traceback.format_exception_only(type(e), e)
                if "DataX2" in t_x2[0]:
                    self.cnt2 += 1
        
        except AttributeError as e:
            t_x2 = traceback.format_exception_only(type(e), e)
            if "DataY" in t_x2[0]:
                messagebox.showerror('警告', "目的変数Yが選択されていません\n"+t_x2[0])
                self.cnt2 += 1
            elif "DataX" in t_x2[0]:
                messagebox.showerror('警告', "説明変数Xが選択されていません\n"+t_x2[0])
                self.cnt2 += 1
            else:
                 messagebox.showerror('警告', "データに異常が発生しました。\n"+t_x2[0])
                 self.cnt2 += 1                 
            
        if self.cnt2 == 0:
            box7.delete(0, tk.END)
            box7.insert(tk.END, f"「{file_name}」ファイルが正しく挿入されました。")  

        elif self.cnt_fnfe_x2 != 0:
            box7.delete(0, tk.END)
            box7.insert(tk.END, "ファイルが選択されていません")
            
        else:
            box7.delete(0, tk.END)
            box7.insert(tk.END, "【エラー】データを見直して下さい。")

In [98]:
class Gui:

    def __init__(self):
        data_with_model_new = 0
        y_test = 0
        x_test = 0
        trained_model = 0
        learning_log = 0
        data_with_model = 0
        
    def train_model(self):
    
        time.sleep(3)
    #どういう条件を当てはめた方がいいのか
        #XとX2の列名があってなくても列の数だけあっていれば動く
        try:
            columns_y = data_y.columns_y
            df_x = data_x.df_x
            df_y = data_y.df_y
            
            if (len(columns_y) == 1) and (len(df_x)==len(df_y)):
                x = np.array(df_x[:])
                y = np.array(df_y[:])
                #global y_test,x_test
                x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0, shuffle = True)
                x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=0, shuffle = True)
            # モデルの初期化
            
                time.sleep(5)
                
                #global trained_model
                
                model = keras.Sequential()
                # 入力層
                model.add(Dense(12, activation='relu', input_shape=(12,), kernel_regularizer=regularizers.l2(0.01)))
                # 出力層
                model.add(Dense(1, activation='sigmoid'))
                # モデルの構築
                model.compile(optimizer = "adam", loss='binary_crossentropy', metrics=['accuracy'])
                trained_model = model
                
                time.sleep(3)
                
                # 学習の実施
                log = model.fit(x_train, y_train, epochs=1000, batch_size=32, verbose=True,
                            callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                                     min_delta=0, patience=20,
                                                                     verbose=1)],validation_data=(x_valid, y_valid))
                #global learning_log
                learning_log = log
            
                time.sleep(5)
                valid_loss, valid_acc = model.evaluate(x_valid, y_valid, verbose=0)
                
                box5.insert(tk.END, "学習が完了しました") 
            else:
                box5.insert(tk.END, "問題が発生しました。行数があっているか、データが入力されているか確認してください")
                
        except AttributeError as e:
            t_gui_attribute_error = traceback.format_exception_only(type(e), e)
            if "DataX" in t_gui_attribute_error[0]:
                messagebox.showerror('警告', "説明変数Xに異常があります。データを見直して下さい。\n"+t_gui_attribute_error[0])
                box5.delete(0, tk.END)
                box5.insert(tk.END, "データを見直して下さい。") 
            elif "DataY" in t_gui_attribute_error[0]:
                messagebox.showerror('警告', "目的変数Yに異常があります。データを見直して下さい。\n"+t_gui_attribute_error[0])
                box5.delete(0, tk.END)
                box5.insert(tk.END, "データを見直して下さい。") 
            else:
                messagebox.showerror('警告', "異常があります。データを見直して下さい。\n"+t_gui_attribute_error[0])
                box5.delete(0, tk.END)
                box5.insert(tk.END, "データを見直して下さい。") 
                    
    def show_accuracy_score(self):
        y_pred = trained_model.predict(x_test)
        # 二値分類は予測結果の確率が0.5以下なら0,
        # それより大きければ1となる計算で求める
        time.sleep(2)
        y_pred_probability = (y_pred > 0.5).astype("int32")
        y_pred_ = y_pred_probability.reshape(-1)
        cr = classification_report(y_test, y_pred_)
        box6.insert('1.0',cr) 
        
    def analyze_newdata(self):
        if set(columns) == set(columns_x2):
            x_test2 = np.array(df_x2[:])
            y_pred2 = trained_model.predict(x_test2)
            # 二値分類は予測結果の確率が0.5以下なら0,
            # それより大きければ1となる計算で求める    
            time.sleep(2)
            nd_data = np.concatenate([x_test2, y_pred2], axis=1)
            #global data_with_model
            data_with_model = pd.DataFrame(nd_data)
            time.sleep(2)
            #columns = df_X2.columns.append("independent variable")
            #data_with_model = data_with_model.set_axis([columns], axis=1)
            box9.insert(tk.END,"予測が完了しました")
        else:
            box9.insert(tk.END,"学習データと列名が一致しません。データを見直して下さい。")
        
    #Y_pred2の数値をもとにリストを昇順。
    #いくつかのレベルに分けるかグラデーションか選べるようにする
    #ドロップダウンリストも作る
    
    def color_highlight(x):
        if x[12]>0.8:        
            return['background-color: #c30010']*len(data_with_model.columns)
        elif x[12]>0.6:
            return['background-color: #f45d75']*len(data_with_model.columns)
        elif x[12]>0.4:
            return['background-color: #f7b4bb']*len(data_with_model.columns)
        elif x[12]>0.2:
            return['background-color: #f7d7dc']*len(data_with_model.columns)
        else:
            return['background-color: white']*len(data_with_model.columns)
        
    def highlight_rows(self):
        #global data_with_model_new
        data_with_model_new = data_with_model.style.apply(color_highlight, axis = 1) 
        time.sleep(2)
        box10.insert(tk.END,"データの成型が完了しました")    
        
    def to_excel(self):
        data_with_model_new.to_excel("example_dataframe.xlsx")
        time.sleep(2)
        box11.insert(tk.END,"エクセルファイルの出力が完了しました")    


In [ ]:
#ウィンドウのレイアウトを作る
#ボタンを機能させる
#ボタンやボックスのレイアウト

data_x = DataX()
data_y = DataY()
data_x2 = DataX2()
gui = Gui()

main_win = tk.Tk()
main_win.title("簡易ディープラーニングツール")
main_win.geometry("600x500")
box1 = tk.Entry(width=40, text = "xlsxファイルのフルパス")
box1.place(x=20,y=30)
box1_label = tk.Label(text = "説明変数データX")
box1_label.place(x=270,y=30)
box1.insert(tk.END, "ファイルを選択してください")
set_button = tk.Button(text = "ファイルを選択(xlsxファイル)", command = data_x.set)
set_button.place(x = 419, y = 27)

box2 = tk.Entry(width=40)
box2.place(x=20,y=70)
box2_label = tk.Label(text = "目的変数データY")
box2_label.place(x=270,y=70)
box2.insert(tk.END, "ファイルを選択してください")
set_button = tk.Button(text = "ファイルを選択(xlsxファイル)", command = data_y.set)
set_button.place(x = 419, y = 67)

box5 = tk.Entry(width=40)
box5.place(x=20,y=110)
box5_label = tk.Label(text = "完了ステータス")
box5_label.place(x=270,y=110)
set_button = tk.Button(text = "機械学習モデルを構築", command= gui.train_model)
set_button.place(x = 419, y = 107)

box6 = tk.Text()
box6.pack()
box6.place(x=20,y=160, width=380,height=120)
set_button = tk.Button(text = "モデルの予測精度を表示", command = gui.show_accuracy_score)
set_button.place(x = 419, y = 197)

box7 = tk.Entry(width=40)
box7.place(x=20,y=310)
box7_label = tk.Label(text = "説明変数データX2")
box7_label.place(x=270,y=310)
box7.insert(tk.END, "ファイルを選択してください")
set_button = tk.Button(text = "新しいファイルを選択(xlsxファイル)", command = data_x2.set)
set_button.place(x = 419, y = 307)

box9 = tk.Entry(width=40)
box9.place(x=20,y=350)
box9_label = tk.Label()
box9_label.place(x=270,y=350)
set_button = tk.Button(text = "新しいデータを予測", command = gui.analyze_newdata)
set_button.place(x = 419, y = 347)

box10 = tk.Entry(width=40)
box10.place(x=20,y=390)
box10_label = tk.Label()
box10_label.place(x=270,y=390)
set_button = tk.Button(text = "データを成型", command = gui.highlight_rows)
set_button.place(x = 419, y = 387)

box11 = tk.Entry(width=40)
box11.place(x=20,y=430)
box11_label = tk.Label()
box11_label.place(x=270,y=430)
set_button = tk.Button(text = "エクセルファイルを出力", command = gui.to_excel)
set_button.place(x = 419, y = 427)

main_win.mainloop()